<a href="https://colab.research.google.com/github/cojocarucosmin/Optimizers/blob/main/AI_Logistics_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AI Logistics Price Prediction Model**

In [4]:
!pip install -q shap gradio matplotlib numpy pandas scikit-learn

In [5]:
# Logistics Price AI  ·  lightweight two-tab demo
import gradio as gr, pandas as pd, numpy as np, shap, tempfile, matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ───────────── helpers ──────────────
def meta_map_from_csv(meta_file):
    if meta_file is None:
        return {}
    m = pd.read_csv(meta_file.name).fillna("")
    m["label"] = m["section"] + " | " + m["column"] + " | " + m["frequency"]
    return dict(zip(m["column"], m["label"]))

def save_summary_small(rf, X, meta_map):
    Xn = X.copy(); Xn.columns = [meta_map.get(c, c) for c in X.columns]
    expl = shap.TreeExplainer(rf)
    plt.figure(figsize=(4, 3))
    shap.summary_plot(expl.shap_values(Xn), Xn, show=False)
    tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    plt.savefig(tmp.name, bbox_inches="tight"); plt.close()
    return tmp.name

def small_waterfall_png(rf, X_row, meta_map):
    expl = shap.TreeExplainer(rf)
    sv   = expl.shap_values(X_row)[0]
    names= [meta_map.get(c, c) for c in X_row.columns]
    expln= shap.Explanation(values=sv, base_values=expl.expected_value,
                            data=X_row.values[0], feature_names=names)
    ax   = shap.plots.waterfall(expln, show=False)
    fig  = ax.figure if hasattr(ax, "figure") else plt.gcf()
    tmp  = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    fig.savefig(tmp.name, bbox_inches="tight"); plt.close(fig)
    return tmp.name

# ───────── train / load (in-memory) ─────────
def train_model(train_csv, meta_csv, state):
    if train_csv is None:
        return "⚠️ Upload training CSV.", None, {}
    df = pd.read_csv(train_csv.name)
    if "client_name" in df: df = df.drop("client_name", axis=1)
    if "price_total" not in df:
        return "`price_total` column missing.", None, {}

    X, y = df.drop("price_total", axis=1).fillna(0), df["price_total"]
    Xt, Xv, yt, yv = train_test_split(X, y, test_size=.2, random_state=42)
    rf = RandomForestRegressor(n_estimators=120, random_state=42).fit(Xt, yt)

    mae  = mean_absolute_error(yt, rf.predict(Xt))
    rmse = mean_squared_error(yv, rf.predict(Xv))**0.5
    r2   = rf.score(Xv, yv)
    msg  = f"**MAE {mae:.1f} | RMSE {rmse:.1f} | R² {r2:.2f}**"

    meta_map = meta_map_from_csv(meta_csv)
    summ_png = save_summary_small(rf, X, meta_map)
    state["model"], state["features"], state["meta"] = rf, X.columns.tolist(), meta_map
    return msg, summ_png, state

# ───────── prediction ──────────
def predict(offer_csv, state):
    if offer_csv is None or "model" not in state:
        return pd.DataFrame(), gr.update(choices=[]), state
    df = pd.read_csv(offer_csv.name).fillna(0).reset_index(drop=True)
    rf, feats = state["model"], state["features"]
    df["predicted_price"] = np.round(rf.predict(df[feats]), 2)

    df_view = df.copy()
    for c in df_view.columns:
        if c != "client_name":
            df_view[c] = df_view[c].apply(lambda x: f"{x:,.2f}"
                                          if c == "predicted_price"
                                          else f"{x:,}")
    names = df_view.get("client_name",
                        pd.Series([f"Offer {i+1}" for i in range(len(df_view))]))
    choices = [f"{i}: {n}" for i, n in enumerate(names)]
    state["Xoffers"] = df[feats].to_dict()
    return df_view, gr.update(choices=choices, value=None), state

# ───────── individual SHAP ──────────
def explain(row_choice, state):
    if not row_choice or "Xoffers" not in state:
        return None
    idx = int(row_choice.split(":")[0])
    X   = pd.DataFrame(state["Xoffers"]).reset_index(drop=True)
    return small_waterfall_png(state["model"], X.iloc[[idx]], state["meta"])

# ─────────────── UI ──────────────
with gr.Blocks(title="Logistics Price AI") as demo:
    gr.Markdown("## AI Logistics Price Prediction Model")

    app_state = gr.State({})          # lives for the session

    # tab 1
    with gr.Tab("Train / Load"):
        tr_csv  = gr.File(label="Training CSV")
        meta_csv= gr.File(label="Metadata catalog (optional)")
        tr_btn  = gr.Button("Train model", variant="primary")
        tr_msg  = gr.Markdown()
        tr_img  = gr.Image(type="filepath")

    # tab 2
    with gr.Tab("Predict & Explain"):
        off_csv = gr.File(label="Offer(s) CSV")
        pr_btn  = gr.Button("Predict", variant="primary")
        table   = gr.Dataframe()
        dd      = gr.Dropdown(label="Pick offer row")
        sh_img  = gr.Image(type="filepath")

    # wiring
    tr_btn.click(train_model, [tr_csv, meta_csv, app_state],
                 [tr_msg, tr_img, app_state])

    pr_btn.click(predict, [off_csv, app_state],
                 [table, dd, app_state])

    dd.change(explain, [dd, app_state], sh_img)

demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3f2f73270f2c326832.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3f2f73270f2c326832.gradio.live
